In [3]:
from dotenv import load_dotenv
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_pinecone import PineconeVectorStore

load_dotenv()

True

## 파일로드

In [5]:
from langchain_community.document_loaders import CSVLoader

dur_con_med = CSVLoader('./DUR_con_med.csv')
dur_age = CSVLoader('./DUR_age.csv')
dur_old = CSVLoader('./DUR_older.csv')
dur_pregnant = CSVLoader('./DUR_pregnant.csv')

In [6]:
con_med_docs = dur_con_med.load()
age_docs = dur_age.load()
old_docs = dur_old.load()
pregnant_docs = dur_pregnant.load()

In [12]:
import random
con_med_docs = random.sample(con_med_docs, 100)
age_docs = random.sample(age_docs, 100)
old_docs = random.sample(old_docs, 100)
pregnant_docs = random.sample(pregnant_docs, 100)

In [14]:
documents = con_med_docs + age_docs + old_docs + pregnant_docs

In [15]:
print(type(documents))
print(len(documents))

<class 'list'>
400


## 임베딩 모델 생성

In [16]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [17]:
import os
from langchain_pinecone import PineconeVectorStore

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [29]:
print(PINECONE_INDEX_NAME)
print(PINECONE_NAMESPACE)
print(PINECONE_API_KEY)

ai-medicine-rag
ai-medicine-rag-ns
pcsk_4XyuMX_5f47dcAxtKB3mkLzQbc4EfucgVHPSkZ7pikmbjCRbviWzyUBGstWunj4XTVTmaj


## 의약품 설명(Prompt)

In [133]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def medicine_info(query):
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''당신은 전문 약사입니다. 사용자가 약물 성분에 대해 질문하면, 아래의 기준에 따라 친절하고 신뢰성 있는 답변을 제공합니다. 질문에 따라 적절한 세부 정보를 포함하되, 지나치게 복잡한 전문 용어는 쉽게 풀어서 설명합니다. 필요한 경우 사용자에게 추가로 주의 사항을 알려줍니다.

    1. 성분명 및 기본 정보:
    - 해당 성분의 이름(화학명 및 일반명)을 정확히 설명하세요.
    - 이 성분이 포함된 대표적인 약물을 예로 들어 설명합니다.

    2. 효능/효과:
    - 이 성분이 치료 또는 완화에 사용되는 질환/증상을 설명합니다.
    - 일반적인 용도를 간결하게 안내합니다.

    3. 작용 기전:
    - 이 성분이 신체에서 어떻게 작용하는지 간단히 설명합니다.
    - 너무 복잡한 메커니즘 대신, 쉽게 이해할 수 있는 표현을 사용하세요.

    4. 부작용:
    - 일반적인 부작용(발생 가능성이 높은 증상)을 명시하세요.
    - 드물지만 심각한 부작용도 언급하며, 발생 시 대처 방법을 알려줍니다.

    5. 복용 및 주의사항:
    - 올바른 복용 방법(용량, 시간 등)을 안내합니다.
    - 음식, 다른 약물, 알코올 등과의 상호작용 여부를 명확히 설명합니다.
    - 임신, 수유 중 복용 가능 여부를 포함합니다.

    6. 상호작용:
    - 이 성분이 특정 약물이나 음식과 상호작용할 가능성이 있는 경우를 설명합니다.

    7. 금기 사항:
    - 이 성분을 복용해서는 안 되는 특정 조건(알레르기, 특정 질환 등)을 명시합니다.

    8. 추가 권장 사항:
    - 사용자가 혼란스러울 수 있는 부분에 대해 명확히 하고, 필요시 의사나 약사와 상담을 권합니다.


    예시 프롬프트
    질문: "이부프로펜 성분에 대해 알려주세요."

    답변:  
    "안녕하세요, 저는 약사입니다. 이부프로펜에 대해 설명드릴게요.  
    1. 성분명: 이부프로펜은 비스테로이드성 소염진통제(NSAID)로, 흔히 사용하는 진통제 및 소염제입니다.  
    2. 효능/효과: 주로 통증 완화(두통, 치통, 생리통), 염증 완화, 발열 감소에 사용됩니다.  
    3. 작용 기전: 이부프로펜은 체내 염증 매개 물질인 프로스타글란딘의 생성을 억제하여 통증과 염증을 줄입니다.  
    4. 부작용 위장 장애(속 쓰림, 위염), 드물게 위궤양, 피부 발진 등이 발생할 수 있습니다.  
    5. 복용 및 주의사항 식사 후에 복용하는 것이 좋으며, 공복에 복용하면 위장 장애를 유발할 수 있습니다.  
    6. 상호작용: 아스피린, 와파린 같은 항응고제와 함께 복용하면 출혈 위험이 증가할 수 있으니 주의하세요.  
    7. 금기 사항: 소화성 궤양이 있거나 NSAID에 과민반응을 보인 적이 있는 분은 복용하지 마세요.  
    8. 추가 권장 사항: 복용 중 이상 증상이 나타나면 즉시 의사나 약사와 상담하세요."

    유의사항
        - 약물 성분 정보는 최신 지침을 바탕으로 작성되어야 하며, 신뢰할 수 있는 데이터베이스(예: Micromedex, UpToDate, 약학정보원 등)를 활용해 근거를 제공해야 합니다.
        - 사용자의 질문을 명확히 파악하여, 필요 이상으로 과도한 정보는 생략하고 핵심만 전달합니다.

        '''),
        ('user', '이미지의 약에 대해 설명해줘'),
    ])
    prompt += HumanMessagePromptTemplate.from_template([query])
    model = ChatOpenAI(
        model='gpt-4o',
        temperature=1
    )
    
    output_parser = StrOutputParser()
    
    chain = prompt | model | output_parser
    return chain

In [134]:
# prompt 확인
chain = RunnableLambda(medicine_info)
chain.invoke({'image_url': 'https://kdpharma.co.kr/uploaded/shop_good/143/0bdcbaa6376b40dbe9c5e10c0e3b26010.jpg'})

'안녕하세요, 이 약은 아세클로페낙 성분을 포함하고 있습니다. 아세클로페낙에 대해 설명드릴게요.\n\n1. 성분명: 아세클로페낙은 비스테로이드성 소염진통제(NSAID)입니다. 염증과 통증을 줄이기 위해 사용됩니다.\n\n2. 효능/효과: 주로 류마티스 관절염, 골관절염, 강직성 척추염 등의 염증성 질환에 사용됩니다.\n\n3. 작용 기전: 아세클로페낙은 염증을 유발하는 물질인 프로스타글란딘의 생성을 억제하여 통증과 염증을 감소시킵니다.\n\n4. 부작용: 위장 장애(속 쓰림, 소화불량), 드물게 간 기능 이상, 피부 발진 등이 발생할 수 있습니다.\n\n5. 복용 및 주의사항: 보통 식사 후에 복용하는 것이 좋습니다. 공복에 복용 시 위장 장애를 유발할 수 있습니다.\n\n6. 상호작용: 아스피린 등의 다른 NSAID, 와파린 같은 항응고제와 함께 복용 시 주의가 필요합니다. 출혈 위험이 증가할 수 있습니다.\n\n7. 금기 사항: 소화성 궤양, 심한 심혈관 질환, 아세클로페낙에 과민반응 이력이 있는 분은 복용하지 않는 것이 좋습니다.\n\n8. 추가 권장 사항: 이상 증상이 나타날 경우 즉시 의사나 약사와 상담하시기 바랍니다.\n\n기타 궁금한 사항이 있으면 언제든지 질문해주세요.'

## VectorDB

In [132]:
from langchain_core.runnables import RunnableLambda
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os


PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

def similarity_search_medicine(query):

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,   
        pinecone_api_key=PINECONE_API_KEY
)
    
    result = vector_db.similarity_search(
        query,
        k=1,
        namespace=PINECONE_NAMESPACE
    )
    return{
        "medicine_name": query,
        "medicine_content" : '\n'.join([doc.page_content for doc in result])
    }

    

In [129]:
# vectordb 결과 확인
chain = RunnableLambda(similarity_search_medicine)

print(chain.invoke('에이서캡슐'))

{'medicine_name': '에이서캡슐', 'medicine_ingredients': '성분명: tegafur+gimeracil+oteracil\n성분코드: 452400ACH\n제품코드: 649807210\n제품명: 테고캡슐20_(1캡슐)\n업체명: 명문제약(주)\n고시일자: 2016-12-30\n고시번호: 20160155\n금기등급: 2\n상세정보: 기형아 출산 보고.동물실험에서 태아의 내장이상, 골격이상, 화골지연 등 보고.'}


In [ ]:
# 이미지 url
# https://kdpharma.co.kr/uploaded/shop_good/143/0bdcbaa6376b40dbe9c5e10c0e3b26010.jpg

In [135]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(medicine_info) | RunnableLambda(similarity_search_medicine)

In [137]:
# 최종 결과 확인
print(chain.invoke({'image_url': 'https://kdpharma.co.kr/uploaded/shop_good/143/0bdcbaa6376b40dbe9c5e10c0e3b26010.jpg'}))

{'medicine_name': '안녕하세요, 저는 약사입니다. 이 캡슐에 대해 설명드릴게요.\n\n이 약은 "에이서 캡슐"로 "아세클로페낙" 성분이 포함된 약입니다.\n\n1. **성분명**: 아세클로페낙은 비스테로이드성 소염진통제(NSAID)입니다. \n\n2. **효능/효과**: 주로 관절염이나 근육 통증과 같은 염증성 질환의 통증 완화에 사용됩니다.\n\n3. **작용 기전**: 이 약물은 체내 염증 물질인 프로스타글란딘의 생성을 억제하여 통증과 염증을 줄입니다.\n\n4. **부작용**: 위장 장애(속 쓰림, 위염), 드물게 위궤양, 피부 발진 등의 부작용이 발생할 수 있습니다.\n\n5. **복용 및 주의사항**: 식사 후에 복용하는 것이 좋습니다. 공복에 복용하면 위장 장애가 생길 수 있습니다.\n\n6. **상호작용**: 다른 NSAID나 항응고제와 함께 복용 시 출혈 위험이 증가할 수 있으니 주의해야 합니다.\n\n7. **금기 사항**: 아세클로페낙에 과민반응이 있거나 소화성 궤양 환자는 복용하지 않는 것이 좋습니다.\n\n8. **추가 권장 사항**: 이상 반응이 나타나면 즉시 의사나 약사와 상담하시기 바랍니다.\n\n더 궁금한 점이 있으시면 알려주세요.', 'medicine_content': '성분명: aceclofenac\n성분코드: 100901ACH\n제품코드: 648101510\n제품명: 에이서캡슐(아세클로페낙)_(0.1g/1캡슐)\n업체명: 경동제약(주)\n고시일자: 2016-12-30\n고시번호: 20160155\n금기등급: 2\n상세정보: 임부에 대한 안전성 미확립.임신 말기에 투여시 태아의 동맥관조기폐쇄 가능성.동물실험에서 비스테로이드성 소염진통제는 난산발생빈도 증가, 분만지연, 태아 생존율 감소 보고.임신 약 20주 이후 비스테로이드성 소염제의 사용은 태아의 신기능 이상을 일으켜 양수 과소증 유발 가능 및 경우에 따라 신생아 신장애 발생 가능'}
